In [2]:
addprocs(40)

IF_files = String[]
for (root, dirs, files) in walkdir("/home/hdd1/YaoLu/train/")
    for file in files
        file[1:2] == "IF" && push!(IF_files,joinpath(root, file))
    end
end

@everywhere begin

function profit(signal,buy_price,sell_price)
    count = 0
    state = 0
    price = 0
    profits = 0
    for t=1:length(signal)
        if signal[t] == 1   
            if state == 0
                adv_price = sell_price[t]   
                state = 1
                price = adv_price              
            elseif state == -1
                adv_price = sell_price[t]
                profits += price - adv_price - 0.4
                count += 1
                state = 0
                price = 0              
            end
        elseif signal[t] == -1
            if state == 0
                adv_price = buy_price[t]   
                state = -1
                price = adv_price
            elseif state == 1
                adv_price = buy_price[t]
                profits += -price + adv_price - 0.4
                count += 1
                state = 0
                price = 0
            end
        end
    end
    return count,profits
end 
    
function myconv(x,y)
    nx = length(x)
    ny = length(y)
    nz = nx + ny -1
    z = zeros(nz)
    for n in 0:nz-1
        z[n+1] = sum(x[n-m+1]*y[m+1] for m=max(n-nx+1,0):min(ny-1,n))
    end
    z[1:ny]
end

function profit(w,file::String)
    name = tempname()
    write(name,String(read(file)[253:end]))
    data = readcsv(name);
    data[data.==""] = 0.0
    bp = Float64.(data[9:end-10,18])
    bq = Float64.(data[9:end-10,19])
    sp = Float64.(data[9:end-10,20])
    sq = Float64.(data[9:end-10,21])
    p = (bp.*bq + sp.*sq)./(bq + sq)
    q = Float64.(data[9:end-10,10])
    dp = diff(p)
    dq = diff(q)
    qp = dq.*dp

    cqp = myconv(w[1],qp)
            signal = (abs(cqp).>w[2]).*sign(cqp)
    profit([0;signal],bp,sp)
end

end #end of @everywhere
    
function profit(w,files::Vector{String})
    profits = @parallel (vcat) for i = 1:length(files)
     profit(w,files[i])[2]
    end
    return mean(profits)
end

 at In[1]:13 overwritten at In[2]:13.
 overwritten at In[2]:13.


profit (generic function with 3 methods)

In [3]:
w = Any[ones(30)/30,0.1]
@time profit(w,IF_files[1])
@time profit(w,IF_files[1:30])

  3.266408 seconds (11.03 M allocations: 306.564 MB, 7.91% gc time)
  6.846910 seconds (606.30 k allocations: 25.178 MB, 0.23% gc time)


-674.1866666666645

In [ ]:
for θ=0:0.01:2
    w = Any[ones(30)/30,θ]
    @show θ,profit(w,IF_files[1:80])
end

(θ,profit(w,IF_files[1:80])) = (0.0,-1380.1450000000425)
(θ,profit(w,IF_files[1:80])) = (0.01,-1229.9525000000226)
(θ,profit(w,IF_files[1:80])) = (0.02,-1122.0475000000113)
(θ,profit(w,IF_files[1:80])) = (0.03,-1037.1325000000047)
(θ,profit(w,IF_files[1:80])) = (0.04,-966.5325000000005)
(θ,profit(w,IF_files[1:80])) = (0.05,-907.1699999999988)


fatal: error thrown and no exception handler available.
InterruptException()
jl_run_once at /home/centos/buildbot/slave/package_tarball64/build/src/jl_uv.c:142
process_events at ./libuv.jl:82
wait at ./event.jl:147
task_done_hook at ./task.jl:191
unknown function (ip: 0x7f24a80cdf02)
jl_call_method_internal at /home/centos/buildbot/slave/package_tarball64/build/src/julia_internal.h:189 [inlined]
jl_apply_generic at /home/centos/buildbot/slave/package_tarball64/build/src/gf.c:1942
jl_apply at /home/centos/buildbot/slave/package_tarball64/build/src/julia.h:1392 [inlined]
finish_task at /home/centos/buildbot/slave/package_tarball64/build/src/task.c:214 [inlined]
start_task at /home/centos/buildbot/slave/package_tarball64/build/src/task.c:261
unknown function (ip: 0xffffffffffffffff)
ErrorException

(θ,profit(w,IF_files[1:80])) = (0.06,-857.7049999999987)
